In [ ]:
# !pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.webdriver import WebDriver
from bs4 import BeautifulSoup
import requests


In [ ]:
driver = webdriver.Chrome()
# Islamabad Page
driver.get('https://www.zameen.com/new-projects/islamabad-3-1/')


# Get all the anchor tags
all_projects_selector = 'main > div > div > div > div > div > a'

elements = driver.find_elements(By.CSS_SELECTOR, all_projects_selector)


In [ ]:
type(driver)

##### Loop through Tags and extract the links

In [ ]:
def html_to_file(html, output_file = 'output.html'):
    pretty_soup = BeautifulSoup(html, 'html.parser').prettify()
    # Write the BeautifulSoup object to an HTML file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(pretty_soup)

1. Get Links for all the pages projects in the City
    TO Do this, Open the city page, get all the links,
    Do this till gone through all the pages of a city.
2. Then open these links in selenium one by page, and gather information for each project

In [ ]:
url = f'https://www.zameen.com/new-projects/islamabad-3-1/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

offering_divs = soup.select("main > div > div > div > div > div > a")

offering_divs


In [ ]:
urls = [ele.get('href') for ele in offering_divs]

In [ ]:
class City:
    def __init__(self, url) -> None:
        self.url = url
    
    def _get_total_pages(self):
        response = requests.get(self.url)
        soup = BeautifulSoup(response.text)

        total_pages = int(soup.select_one('ul[aria-label="Pagination"] > li:nth-last-child(2) > a').text)

        return total_pages

    def get_all_project_links(self):
        
        city_project_urls = []
        total_pages = self._get_total_pages()

        for page in range(1, total_pages+1):
            response = requests.get(f'https://www.zameen.com/new-projects/islamabad-3-1/?page={page}')
            soup = BeautifulSoup(response.text)
            proj_tags = soup.select("main > div > div > div > div > div > a")
            urls = [ele.get('href') for ele in proj_tags]
            city_project_urls.extend(urls)

        return city_project_urls
    
    def scrap(self):
        pass
        

In [3]:
city_soup = BeautifulSoup(requests.get('https://www.zameen.com/new-projects/islamabad-3-1/').text)

In [4]:
# There is only a single ul having area-label of Pagination
total_pages = int(city_soup.select_one('ul[aria-label="Pagination"] > li:nth-last-child(2) > a').text)
total_pages

24

In [5]:
range(1, total_pages+1)

range(1, 25)

In [6]:
city_project_urls = []
for page in range(1, total_pages+1):
    response = requests.get(f'https://www.zameen.com/new-projects/islamabad-3-1/?page={page}')
    soup = BeautifulSoup(response.text)
    proj_tags = soup.select("main > div > div > div > div > div > a")
    urls = [ele.get('href') for ele in proj_tags]
    city_project_urls.extend(urls)

In [7]:
city_project_urls

['/new-projects/grand_orchard-2275/',
 '/new-projects/airport_enclave_villas-2434/',
 '/new-projects/360_the_residences-2241/',
 '/new-projects/pearl_square_residency-2302/',
 '/new-projects/rjs_lifestyle_residences-2024/',
 '/new-projects/rjs_square-2269/',
 '/new-projects/sparco_tower-2280/',
 '/new-projects/jasmine_suites-2296/',
 '/new-projects/river_hills_5-2226/',
 '/new-projects/nova_city-2250/',
 '/new-projects/hyde_park_one-2148/',
 '/new-projects/grande_palladium-2090/',
 '/new-projects/roman_grove-1940/',
 '/new-projects/zameen_ace_mall-1623/',
 '/new-projects/zameen_ace_homes-1694/',
 '/new-projects/_safa_burj_mall_-476/',
 '/new-projects/ideas_one-1076/',
 '/new-projects/pavilion_17-2428/',
 '/new-projects/tower_4-2338/',
 '/new-projects/nora_residences-2348/',
 '/new-projects/cloud_tower_1-2255/',
 '/new-projects/al_karim_residence-2233/',
 '/new-projects/royal_mall_and_residency-913/',
 '/new-projects/citadel_7-2205/',
 '/new-projects/zeta_1_mall-1891/',
 '/new-projects/

In [ ]:
class Page:
    def __init__(self, url) -> None:
        self.url = url

    def scrap(self, driver: WebDriver):
        driver.get(url)

        self._get_overview(driver)

    
    def _get_overview(self, driver):
        overview_elements = driver.find_elements(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(1) > div > div[class^="new-projects-overview"] [class^="new-projects-overview_value"]')

        offerings = overview_elements[0].text
        offerings = list(map(lambda x: x.strip(), offerings.split(' ')))


        developer = overview_elements[1].text
    




    

In [ ]:
html_to_file(city_soup.select_one('ul[aria-label="Pagination"]').prettify(), 'pagi.html')

In [23]:
driver = webdriver.Chrome()

proj_base_url = 'https://www.zameen.com'
page_url = proj_base_url + city_project_urls[0]
driver.get(page_url)


In [24]:
# driver.find_elements(By.CSS_SELECTOR, 'h2[class*="heading"]')
overview_elements = driver.find_elements(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(1) > div > div[class^="new-projects-overview"] [class^="new-projects-overview_value"]')

offerings = overview_elements[0].text
developer = overview_elements[1].text

In [25]:
list(map(lambda x: x.strip(), offerings.split(' ')))

['Flats,', 'Shops']

In [26]:
info_parent_element = driver.find_element(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(2) > div > div')
info_parent_element.get_attribute('outerHTML')

'<div class="rc-collapse property-type-collpase_collapseProjects__1EAfU" style="margin-inline:"><div class="rc-collapse-item rc-collapse-item-active" style="border-radius:"><div class="rc-collapse-header" role="button" tabindex="0" aria-expanded="true"><div class="property-type-collpase_panelHeader__2EKWc"><div class="space_space__2CSKB space_align-center__2jK7Q" style="gap:16px;flex-wrap:"><div><svg width="1em" height="1em" viewBox="0 0 24 24" class="property-type-collpase_icon__2Wg92 Icon_icon__2_bai" style="--icon-size:1.715em"><path d="m11.228 4.278 8.242-1.261c.383-.06.773.045 1.065.294.293.257.465.627.465 1.013v15.92a.755.755 0 0 1-.75.756h-2.438v-3.967c0-.733-.585-1.322-1.312-1.322h-1.875c-.727 0-1.312.59-1.312 1.322V21h-3.188V5.586c0-.65.465-1.202 1.103-1.308Zm5.647 9.733H18c.31 0 .563-.254.563-.567a.565.565 0 0 0-.563-.567h-1.125a.565.565 0 0 0-.562.567c0 .313.252.567.562.567Zm0-2.267H18c.31 0 .563-.254.563-.567A.565.565 0 0 0 18 10.61h-1.125a.565.565 0 0 0-.562.567c0 .313.252

In [27]:
info_elements = info_parent_element.find_elements(By.CSS_SELECTOR, ':scope > div')
for ele in info_elements:
    ele.find_elements()
    # print(ele.get_attribute('outerHTML'))

In [28]:
print(info_elements[0].find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_headerTitle"]').text)
print(info_elements[0].find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_price"]').text)

Flats
PKR 1.03 Crore to 10.22 Crore


In [ ]:
class ElementData:
    
    @staticmethod
    def extract_flats_data():
        

In [29]:
this_project_data = {}

overview_elements = driver.find_elements(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(1) > div > div[class^="new-projects-overview"] [class^="new-projects-overview_value"]')

offerings = overview_elements[0].text
offerings = list(map(lambda x: x.strip(), offerings.split(' ')))


developer = overview_elements[1].text

this_project_data['offerings'] = offerings
this_project_data['developer'] = developer

info_parent_element = driver.find_element(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(2) > div > div')
info_elements = info_parent_element.find_elements(By.CSS_SELECTOR, ':scope > div')

tile_data = []
for info_ele in info_elements:
    info_ele_data = {}
    # Get type name
    type_name = info_ele.find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_headerTitle"]').text
    
    # Get price range
    price_range = info_ele.find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_price"]').text
    
    #Get the info cards within this type

    info_ele_data['type'] = type_name
    info_ele_data['price_range'] = price_range

    info_cards = info_ele.find_elements(By.CSS_SELECTOR, 'div.rc-collapse-content-box div[class*="card_card"]')
            
    
    # Get info from each card
    cards_data_list = []
    for card in info_cards:
        card_data = {}
        (card_type_ele, card_price_ele) = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')
        card_type, card_price = card_type_ele.text, card_price_ele.text

        card_data['type'] = card_type
        card_data['price'] = card_price

        # Info blocks within a card
        info_blocks = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_infoBlock"]')
        
        # A single key would not be repeated within a block
        for block in info_blocks:
            #property name
            property_name = block.find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_key"]').text
            #property value
            property_val = block.find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_value"]').text

            card_data[property_name] = property_val

        cards_data_list.append(card_data)
    
    info_ele_data['cards'] = cards_data_list

    tile_data.append(info_ele_data)

this_project_data['tiles'] = tile_data


In [30]:
this_project_data

{'offerings': ['Flats,', 'Shops'],
 'developer': 'Premier Choice',
 'tiles': [{'type': 'Flats',
   'price_range': 'PKR 1.03 Crore to 10.22 Crore',
   'cards': [{'type': 'Apartment',
     'price': 'PKR 1.03 Crore to 1.67 Crore',
     'Area Size': '2.23 - 3.61 marla',
     'Bathrooms': '1',
     'Bedrooms': '–'},
    {'type': 'Apartment',
     'price': 'PKR 2.13 Crore to 3.78 Crore',
     'Area Size': '4.61 - 8.2 marla',
     'Bathrooms': '1',
     'Bedrooms': '1'},
    {'type': 'Apartment',
     'price': 'PKR 2.48 Crore to 5.46 Crore',
     'Area Size': '6.31 - 13.87 marla',
     'Bedrooms': '2',
     'Bathrooms': '2'},
    {'type': 'Apartment',
     'price': 'PKR 3.67 Crore to 10.22 Crore',
     'Area Size': '9.31 - 25.95 marla',
     'Bathrooms': '3',
     'Bedrooms': '3'},
    {'type': 'Apartment',
     'price': 'PKR 7.46 Crore to 10.13 Crore',
     'Area Size': '18.94 - 25.72 marla',
     'Bathrooms': '4',
     'Bedrooms': '4'}]},
  {'type': 'Shops',
   'price_range': 'PKR 31.15 Lak

In [39]:
info_elements[1].get_attribute('outerHTML')

'<div class="rc-collapse-item" style="border-radius:"><div class="rc-collapse-header" role="button" tabindex="0" aria-expanded="false"><div class="property-type-collpase_panelHeader__2EKWc"><div class="space_space__2CSKB space_align-center__2jK7Q" style="gap:16px;flex-wrap:"><div><svg width="1em" height="1em" viewBox="0 0 24 24" class="property-type-collpase_icon__2Wg92 Icon_icon__2_bai" style="--icon-size:1.715em"><path d="m20.983 9.051-1.018-4.074a1.124 1.124 0 0 0-1.091-.852H5.127c-.516 0-.966.351-1.092.852L3.017 9.05A.55.55 0 0 0 3 9.187C3 10.738 4.198 12 5.672 12c.856 0 1.62-.426 2.11-1.088C8.27 11.574 9.033 12 9.89 12c.856 0 1.62-.426 2.109-1.088.49.662 1.252 1.088 2.11 1.088.857 0 1.62-.426 2.109-1.088.49.662 1.252 1.088 2.11 1.088C19.801 12 21 10.738 21 9.187a.548.548 0 0 0-.017-.136Z" fill="#79D0A4"></path><path d="M18.328 13.124a3.68 3.68 0 0 1-2.11-.661c-1.23.856-2.987.856-4.218 0-1.23.856-2.988.856-4.219 0a3.65 3.65 0 0 1-3.656.312v5.974c0 .621.504 1.125 1.125 1.125h4.5v-4.

#### Extract Flats Data

In [73]:
info_cards = info_elements[0].find_elements(By.CSS_SELECTOR, 'div.rc-collapse-content-box div[class*="card_card"]')
# for card in info_cards:
#     print(card.get_attribute('outerHTML'))

(card_type_ele, card_price_ele) = info_cards[0].find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')
print(card_type_ele.get_attribute('outerHTML'))
driver.execute_script("return arguments[0].textContent;", card_type_ele)
# print(card_type_ele.text, card_price_ele.text)

# info_blocks = info_cards[0].find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_infoBlock"]')
# #property name
# print(info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_key"]').text)
# #property value
# info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_value"]').text

<div><div class="flex flexYcenter" style="gap:10px"><div>Apartment</div></div></div>
Apartment


'Apartment'

#### Extract Shops Data

In [52]:
info_cards = info_elements[1].find_elements(By.CSS_SELECTOR, 'div.rc-collapse-content-box div[class*="card_card"]')
for card in info_cards:
    print(card.get_attribute('outerHTML'))

(card_type_ele, card_price_ele) = info_cards[0].find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')

card_type_ele.text, card_price_ele.text

info_blocks = info_cards[0].find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_infoBlock"]')
#property name
print(info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_key"]').text)
#property value
info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_value"]').text

<div class="card_card__3KEPg property-type-collpase_cardContents__2KvST card_border__2siZ1" style="--padding: 24px;"><div class="w-100 text-dark space_space__2CSKB space_align-center__2jK7Q space_justify-space-between__2vruQ" style="gap: 16px;"><div><div><div class="flex flexYcenter u-mb12" style="gap: 10px;"><div class="fw-700">Commercial outlets Ground Floor</div></div><div class="flex flexYcenter" style="gap: 6px;"><svg width="1em" height="1em" viewBox="0 0 24 24" class="Icon_icon__2_bai" bgStyle="[object Object]" color="#7ed8aa" style="--icon-size: 1em; --icon-color: #7ed8aa; flex: 0 0 auto;"><path d="m20.983 9.051-1.018-4.074a1.124 1.124 0 0 0-1.091-.852H5.127c-.516 0-.966.351-1.092.852L3.017 9.05A.55.55 0 0 0 3 9.187C3 10.738 4.198 12 5.672 12c.856 0 1.62-.426 2.11-1.088C8.27 11.574 9.033 12 9.89 12c.856 0 1.62-.426 2.109-1.088.49.662 1.252 1.088 2.11 1.088.857 0 1.62-.426 2.109-1.088.49.662 1.252 1.088 2.11 1.088C19.801 12 21 10.738 21 9.187a.548.548 0 0 0-.017-.136Z" fill="#79D

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
# html_to_file(requests.get(page_url).text, 'city_page_html.html')